In [1]:
%matplotlib inline

from blockchain import statistics
import requests
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import matplotlib
from sklearn import datasets, linear_model
#import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

<h3> Glossary </h3> 
<p> DT = DAILY TRANSACTIONS</p>
<P> DUA = DAILY UNIQUE ADRESSES </P>
<P> DTV = DAILY TRANSACTION VOLUME </P>
<P> DDTV = DAILY DOLLAR TRANSACTION VOLUME </P>
<p> BIC = BITCOIN IN CIRCULATION </p>
<p> MP = MARKET PRICE </p>

In [3]:
DT_get = requests.get("https://blockchain.info/charts/n-transactions?timespan=all&format=json")
DUA_get = requests.get("https://blockchain.info/charts/n-unique-addresses?timespan=all&format=json")
DTV_get = requests.get("https://blockchain.info/charts/estimated-transaction-volume?timespan=all&format=json")
DDTV_get = requests.get("https://blockchain.info/charts/estimated-transaction-volume-usd?timespan=all&format=json")
BIC_get = requests.get("https://blockchain.info/charts/total-bitcoins?timespan=all&format=json")
MP_get = requests.get("https://blockchain.info/charts/market-price?timespan=all&format=json")

In [4]:
DT_json = DT_get.json()
DUA_json = DUA_get.json()
DTV_json = DTV_get.json()
DDTV_json = DDTV_get.json()
BIC_json = BIC_get.json()
MP_json = MP_get.json()

In [140]:
DT = pd.DataFrame(list(DT_json.values())[1])
DT.columns = ['Date', 'Transactions']

In [154]:
q = DT_json.values()get("day")
q

In [155]:
DT

,Date,Transactions
0,1230940800,1
1,1231113600,0
2,1231286400,0
3,1231459200,14
4,1231632000,106
5,1231804800,116
6,1231977600,136
7,1232150400,109
8,1232323200,120
9,1232496000,115


In [5]:
DT = pd.DataFrame(list(DT_json.values())[1])
DT.columns = ['Date', 'Transactions']

DUA = pd.DataFrame(list(DUA_json.values())[1])
DUA.columns = ['Date', 'UniqueAdresses']
DUA2 = DUA.drop('Date', axis=1)

DTV = pd.DataFrame(list(DTV_json.values())[1])
DTV.columns = ['Date', 'TransactionVolume']
DTV2 = DTV.drop('Date', axis=1)

DDTV = pd.DataFrame(list(DDTV_json.values())[1])
DDTV.columns = ['Date', 'DailyDollarTransactionVolume']
DDTV2 = DDTV.drop('Date', axis=1)

BIC = pd.DataFrame(list(BIC_json.values())[1])
BIC.columns = ['Date', 'BitCoinInCirculation']
BIC2 = BIC.drop('Date', axis=1)

MP = pd.DataFrame(list(MP_json.values())[1])
MP.columns = ['Date', 'MarketPrice']
MP2 = MP.drop('Date', axis=1)
MP['Date'] = pd.to_datetime(MP['Date'],unit='s')

In [6]:
DataSet = pd.concat([DT, DUA2], axis=1)
DataSet = pd.concat([DataSet, DTV2], axis=1)
DataSet = pd.concat([DataSet, DDTV2], axis=1)
DataSet = pd.concat([DataSet, BIC2], axis=1)
DataSet = pd.concat([DataSet, MP2], axis=1)
DataSet['Date'] = pd.to_datetime(DataSet['Date'],unit='s')
DataSet

,Date,Transactions,UniqueAdresses,TransactionVolume,DailyDollarTransactionVolume,BitCoinInCirculation,MarketPrice
0,2009-01-03,1,1,0.000,0.000,50.000,0.000
1,2009-01-05,0,0,0.000,0.000,50.000,0.000
2,2009-01-07,0,0,0.000,0.000,50.000,0.000
3,2009-01-09,14,14,0.000,0.000,750.000,0.000
4,2009-01-11,106,106,0.000,0.000,7600.000,0.000
5,2009-01-13,116,117,10.000,0.000,17800.000,0.000
6,2009-01-15,136,136,414.000,0.000,30450.000,0.000
7,2009-01-17,109,109,100.000,0.000,41650.000,0.000
8,2009-01-19,120,120,150.000,0.000,52650.000,0.000
9,2009-01-21,115,115,500.000,0.000,64100.000,0.000


In [7]:
DataSet = DataSet.drop('DailyDollarTransactionVolume', axis = 1)

In [8]:
DataSet.describe()

,Transactions,UniqueAdresses,TransactionVolume,BitCoinInCirculation,MarketPrice
count,1546.000,1546.000,1546.000,1546.000,1546.000
mean,75565.382,137424.393,173187.967,9695627.579,263.395
std,90722.379,168296.823,245321.945,5127033.777,392.925
min,0.000,0.000,0.000,50.000,0.000
25%,1869.250,2472.500,49823.500,5409487.500,0.790
50%,46869.500,47559.500,143228.500,10961262.500,73.835
75%,106323.000,224307.500,241245.000,14143631.250,427.947
max,367710.000,687290.000,5313161.000,16402262.500,2883.310


In [9]:
DataSet.corr()

,Transactions,UniqueAdresses,TransactionVolume,BitCoinInCirculation,MarketPrice
Transactions,1.000,0.987,0.294,0.798,0.790
UniqueAdresses,0.987,1.000,0.280,0.801,0.818
TransactionVolume,0.294,0.280,1.000,0.352,0.179
BitCoinInCirculation,0.798,0.801,0.352,1.000,0.650
MarketPrice,0.790,0.818,0.179,0.650,1.000


<h1> mmm, outliers could be removed</h1>
<h3> Transactions vs MarketPrice </h3>

In [ ]:
plt.plot(DataSet.Transactions, DataSet.MarketPrice, 'ro')
plt.ylabel('Price (USD)')
plt.xlabel('Transactions Per DAY')
plt.show()

<h3> UniqueAdresses vs MarketPrice</h3>

In [ ]:
plt.plot(DataSet.UniqueAdresses,DataSet.MarketPrice, 'ro')
plt.ylabel('Price (USD)')
plt.xlabel('UniqueAdresses')
plt.show()

<h3> TransactionVolume vs MarketPrice</h3>

In [ ]:
plt.plot(DataSet.TransactionVolume, DataSet.MarketPrice, 'ro')
plt.ylabel('Price (USD)')
plt.xlabel('TransactionVolume')
plt.show()

<h3> BitCoinInCirculation vs MarketPrice</h3>

In [ ]:
plt.plot(DataSet.BitCoinInCirculation, DataSet.MarketPrice, 'ro')
plt.ylabel('Price (USD)')
plt.xlabel('BitCoinInCirculation')
plt.show()

In [12]:
pre_train = DataSet
train_x = DataSet[['Transactions', 'UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
train_y = DataSet['MarketPrice']

In [10]:
train_x = DataSet[['Transactions', 'UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
train_y = DataSet['MarketPrice']

lm1 = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
lm1.fit(train_x,train_y)
yhat = pd.DataFrame(lm1.predict(train_x))
yhat.columns = ['yhat']
lm1.score(train_x, train_y, sample_weight=None)


/usr/local/lib/python3.5/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


0.68230257076947054

In [ ]:
plotter =  pd.concat([train_y, yhat], axis=1)
plotter
plt.plot(plotter.MarketPrice, plotter.yhat, 'ro')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.show()

<h3>7 day forecast</h3>

In [ ]:
train_y_7 = train_y.shift(periods=-7, freq=None, axis=0)
pre_train = pd.concat([train_x,train_y_7], axis = 1).dropna()
train_x = pre_train[['Transactions', 'UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
train_y = pre_train['MarketPrice']

lm1 = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
lm1.fit(train_x,train_y)
yhat = pd.DataFrame(lm1.predict(train_x))
yhat.columns = ['yhat']
lm1.score(train_x, train_y, sample_weight=None)

plotter =  pd.concat([train_y, yhat], axis=1)
plotter
plt.plot(plotter.MarketPrice, plotter.yhat, 'ro')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
lm1.score(train_x, train_y, sample_weight=None),plt.show()

<h3> 30 day forecast </h3>

In [ ]:
train_y_30 = train_y.shift(periods=-30, freq=None, axis=0)
pre_train = pd.concat([train_x,train_y_30], axis = 1).dropna()
train_x = pre_train[['Transactions', 'UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
train_y = pre_train['MarketPrice']

lm1 = linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)
lm1.fit(train_x,train_y)
yhat = pd.DataFrame(lm1.predict(train_x))
yhat.columns = ['yhat']
lm1.score(train_x, train_y, sample_weight=None)

plotter =  pd.concat([train_y, yhat], axis=1)
plotter
plt.plot(plotter.MarketPrice, plotter.yhat, 'ro')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
lm1.score(train_x, train_y, sample_weight=None),plt.show()

In [ ]:
list_y = []
for i in range(0,len(train_y)-30):
    print(train_y[i],train_y[i+30])

<h3> Same but testing other algorithms</h3>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( train_x, train_y, test_size=0.20, random_state=42)

In [ ]:
"""
Boosting Tree Regressor - Note Looking at the analysis this doesn't really look healthy to me. 
Besides the fact that it can now predict higher than 1300 dollar

"""


BTR = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=6, random_state=0, loss='ls').fit(X_train, y_train)
mean_squared_error(y_test, BTR.predict(X_test)) 
BTR.score(X_train, y_train, sample_weight=None),BTR.score(X_test, y_test, sample_weight=None)




In [ ]:
yhat = pd.DataFrame(BTR.predict(X_train))
yhat.columns = ['yhat']
lm1.score(train_x, train_y, sample_weight=None)

plotter =  pd.concat([train_y, yhat], axis=1)
plotter
plt.plot(plotter.MarketPrice, plotter.yhat, 'ro')
plt.ylabel('Predicted')
plt.xlabel('Actual')
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
lm1.score(X_test, y_test, sample_weight=None)

<h1> start here </h1>

In [314]:
pre_train = DataSet
train_x = DataSet[['Transactions', 'UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
train_y = DataSet['MarketPrice']
train_y_30 = train_y.shift(periods=-10, freq=None, axis=0)
pre_train = pd.concat([train_x,train_y,train_y_30], axis = 1).dropna()
pre_train.columns = ['Transactions','UniqueAdresses','TransactionVolume','BitCoinInCirculation','y_now','y_30']

In [316]:
pre_train['y'] = 0
len(pre_train)

1536

In [317]:
for i in range(0,len(pre_train)):
    if pre_train['y_30'][i] > pre_train['y_now'][i]:
        print(i)
        #print(pre_train['y_30'][i],pre_train['y_now'][i])
        pre_train['y'][i] = 1

286
287
288

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



289
290
291
292
293
294
295
300
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
338
349
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
390
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
509
510
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
569
570
571
572
573
574
575
576
587
588
589
590
591
592
593
594
595
596
597
598
599
600
606
607
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
656
663
664
665
667
668
669
670
671
672
673
674
675
676
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
718
722
723
724
725
726
727
728
729
730
731
732
733

In [318]:
pre_train.describe()

,Transactions,UniqueAdresses,TransactionVolume,BitCoinInCirculation,y_now,y_30,y
count,1536.000,1536.000,1536.000,1536.000,1536.000,1536.000,1536.000
mean,74297.680,134748.735,172542.054,9652072.990,247.977,265.110,0.475
std,89618.064,165481.124,245972.955,5115100.622,344.153,393.625,0.500
min,0.000,0.000,0.000,50.000,0.000,0.000,0.000
25%,1809.500,2385.500,49187.000,5360750.000,0.788,0.875,0.000
50%,46422.000,45623.000,142044.000,10915050.000,52.740,88.025,0.000
75%,104557.000,222109.250,240210.250,14089993.750,423.808,428.135,1.000
max,367710.000,687290.000,5313161.000,16364012.500,2387.200,2883.310,1.000


In [319]:
pre_train_x = pre_train[['Transactions','UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
pre_train_y = pre_train['y']

In [320]:
pre_train_x.shape,pre_train_y.shape

((1536, 4), (1536,))

In [321]:
#cant do random sampling here as it is time series
X_train = pre_train_x.loc[0:1443]
X_test = pre_train_x.loc[1444:1543]
y_train = pre_train_y.loc[0:1443]
y_test = pre_train_y.loc[1444:1543]

In [322]:
#X_train, X_test, y_train, y_test = train_test_split( pre_train_x, pre_train_y, test_size=0.20, random_state=42)

In [323]:
I = [5,10,15,20,30,40,50,60,70,80,90,100]
for i in I:
    clf = RandomForestClassifier(n_estimators=i)
    clf.fit(X_train, y_train)
    print(i,clf.score(X_train, y_train),clf.score(X_test, y_test))

5 0.979224376731 0.804347826087
10 0.994459833795 0.804347826087
15 0.997922437673 0.804347826087
20 0.997922437673 0.804347826087
30 0.999307479224 0.804347826087
40 1.0 0.804347826087
50 1.0 0.804347826087
60 1.0 0.804347826087
70 1.0 0.804347826087
80 1.0 0.804347826087
90 1.0 0.804347826087
100 1.0 0.804347826087


In [324]:

clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
print(i,clf.score(X_train, y_train),clf.score(X_test, y_test))

100 0.999307479224 0.804347826087


In [325]:
for_scoring = DataSet[['Transactions','UniqueAdresses','TransactionVolume','BitCoinInCirculation']]
Price = DataSet[['MarketPrice']]
Date = DataSet[['Date']]
x = pd.DataFrame(clf.predict_proba(for_scoring)	)
x.columns = ['sell','buy']
simul = pd.concat([Date,Price,x],axis = 1)

In [326]:
simul
simul = simul[simul['Date'] >= '2017-01-01']

In [327]:
simul

,Date,MarketPrice,sell,buy
1460,2017-01-01,997.720,0.000,1.000
1461,2017-01-03,1023.140,0.000,1.000
1462,2017-01-05,994.670,0.020,0.980
1463,2017-01-07,896.830,0.000,1.000
1464,2017-01-09,894.180,0.000,1.000
1465,2017-01-11,785.220,0.000,1.000
1466,2017-01-13,826.290,0.000,1.000
1467,2017-01-15,822.200,0.020,0.980
1468,2017-01-17,903.840,0.000,1.000
1469,2017-01-19,895.790,0.040,0.960


In [328]:
hold_init = 1000
hold_coins = 1000/simul.MarketPrice[1505]
final = hold_coins*simul.MarketPrice[1545]
final_strat = 14.31*simul.MarketPrice[1545]

In [329]:
starter = 1504
iterator = 10
btc_wallet = 0.0
rand_wallet = 1000.0

for index,row in simul.iterrows():
    print(index,starter,row['Date'],round(btc_wallet,2),round(rand_wallet,2), row['MarketPrice'],row['buy'])
    if index == starter:
        if row['buy'] > 0.5 and rand_wallet != 0:
            btc_wallet = rand_wallet/row['MarketPrice'] - (rand_wallet/row['MarketPrice'])*0.01
            rand_wallet = 0
        if row['buy'] < 0.5 and rand_wallet == 0:
            rand_wallet = btc_wallet*row['MarketPrice'] - (btc_wallet*row['MarketPrice'])*0.01
            btc_wallet = 0
            
        starter = starter + iterator
        
    

1460 1504 2017-01-01 00:00:00 0.0 1000.0 997.72 1.0
1461 1504 2017-01-03 00:00:00 0.0 1000.0 1023.14 1.0
1462 1504 2017-01-05 00:00:00 0.0 1000.0 994.67 0.98
1463 1504 2017-01-07 00:00:00 0.0 1000.0 896.83 1.0
1464 1504 2017-01-09 00:00:00 0.0 1000.0 894.18 1.0
1465 1504 2017-01-11 00:00:00 0.0 1000.0 785.22 1.0
1466 1504 2017-01-13 00:00:00 0.0 1000.0 826.29 1.0
1467 1504 2017-01-15 00:00:00 0.0 1000.0 822.2 0.98
1468 1504 2017-01-17 00:00:00 0.0 1000.0 903.84 1.0
1469 1504 2017-01-19 00:00:00 0.0 1000.0 895.79 0.96
1470 1504 2017-01-21 00:00:00 0.0 1000.0 920.44 0.96
1471 1504 2017-01-23 00:00:00 0.0 1000.0 922.07 0.96
1472 1504 2017-01-25 00:00:00 0.0 1000.0 893.04 1.0
1473 1504 2017-01-27 00:00:00 0.0 1000.0 919.27 0.98
1474 1504 2017-01-29 00:00:00 0.0 1000.0 915.93 0.94
1475 1504 2017-01-31 00:00:00 0.0 1000.0 964.7 0.96
1476 1504 2017-02-02 00:00:00 0.0 1000.0 1007.61 0.96
1477 1504 2017-02-04 00:00:00 0.0 1000.0 1030.99 0.98
1478 1504 2017-02-06 00:00:00 0.0 1000.0 1024.01 0.98

In [330]:
hold_init = 1000
hold_coins = 1000/simul.MarketPrice[1504] - (1000/simul.MarketPrice[1504])*0.01
final = hold_coins*simul.MarketPrice[1545] 

In [331]:
print("new:   :","rand:  ",rand_wallet,"btc(in rand:)  " ,btc_wallet*simul.MarketPrice[1545],"hold:   ", final)

new:   : rand:   0 btc(in rand:)   2627.82570575 hold:    2627.82570575
